<a href="https://colab.research.google.com/github/Lianasy/Decision-Theory-Labs/blob/main/Lab_3_TPR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
#4.1 Обрати варіант та дані температури («Додаток Б»).
# Температури для кожного місяця Каунаса
months_temperature = [-6, -5, 0, 6, 13, 16, 17, 17, 12, 7, 2, -2]


# Дані для елементів одягу
items = {
    'Шапка': {'weight': 1, 'cost': 12},
    'Зимова куртка': {'weight': 4, 'cost': 48},
    'Пухові штани': {'weight': 2, 'cost': 24},
    'Чоботи': {'weight': 2, 'cost': 24},
    'Джинси': {'weight': 1, 'cost': 12},
    'Кросівки': {'weight': 1, 'cost': 12},
    'Светр': {'weight': 1, 'cost': 12},
    'Сорочка': {'weight': 0.5, 'cost': 6},
    'Футболка': {'weight': 0.5, 'cost': 6},
    'Шорти': {'weight': 0.5, 'cost': 6},
    'Куртка': {'weight': 2, 'cost': 24},
    'Пальто': {'weight': 3, 'cost': 36},
    'Рукавиці': {'weight': 0.5, 'cost': 6},
    'Кепка': {'weight': 0.5, 'cost': 6},
    'Черевики': {'weight': 1.5, 'cost': 18},
    'Блайзер': {'weight': 0.5, 'cost': 6},
    'В’єтнамки': {'weight': 0.5, 'cost': 6}
}


# Набори одягу для різних температур з вагою для кожного набору
sets = {
    'Set1': {'temp_range': (-float('inf'), -10), 'items': ['Шапка', 'Зимова куртка', 'Рукавиці', 'Пухові штани', 'Чоботи'], 'weight': 9.5},
    'Set2': {'temp_range': (-9, 0), 'items': ['Шапка', 'Пальто', 'Рукавиці', 'Джинси', 'Чоботи'], 'weight': 7.5},
    'Set3': {'temp_range': (1, 10), 'items': ['Кепка', 'Куртка', 'Джинси', 'Черевики'], 'weight': 5},
    'Set4': {'temp_range': (11, 20), 'items': ['Светр', 'Джинси', 'Кросівки'], 'weight': 3},
    'Set5': {'temp_range': (21, 30), 'items': ['Блайзер', 'Сорочка', 'Джинси', 'Кросівки'], 'weight': 3},
    'Set6': {'temp_range': (30, float('inf')), 'items': ['Блайзер', 'Футболка', 'Шорти', 'В’єтнамки'], 'weight': 2},
}



# Функція для визначення набору одягу для кожного місяця
def get_best_set_for_month(temp, sets):
    for set_name, set_data in sets.items():
        if set_data['temp_range'][0] <= temp <= set_data['temp_range'][1]:
            return set_data['items'], set_name, set_data['weight']
    return [], None, 0

# Функція для визначення недостающих елементів для набору
def calculate_missing_items_cost(available_items, required_items, items):
    missing_items = [item for item in required_items if item not in available_items]
    return sum(items[item]['cost'] + 2 for item in missing_items), missing_items

# Створюємо таблицю витрат
def create_cost_table(sets, months_temperature, items):
    table_data = []

    # Заголовок таблиці
    headers = ['Набір', 'Вартість перевезення'] + [f'Місяць {i+1}' for i in range(12)]

    # Перебір наборів одягу
    for set_name, set_data in sets.items():
        row = [set_name]

        # Розрахунок вартості перевезення
        transportation_cost = set_data['weight'] * 10
        row.append(transportation_cost)

        # Для кожного місяця обчислюємо вартість докупівлі
        for month in range(12):
            temp = months_temperature[month]
            required_items, _, _ = get_best_set_for_month(temp, sets)
            missing_cost, _ = calculate_missing_items_cost(set_data['items'], required_items, items)
            total_cost = missing_cost
            row.append(total_cost)

        table_data.append(row)

    # Формуємо таблицю
    return pd.DataFrame(table_data, columns=headers)

# Створюємо таблицю витрат
cost_table = create_cost_table(sets, months_temperature, items)

# Вивести таблицю
cost_table


,Набір,Вартість перевезення,Місяць 1,Місяць 2,Місяць 3,Місяць 4,Місяць 5,Місяць 6,Місяць 7,Місяць 8,Місяць 9,Місяць 10,Місяць 11,Місяць 12
0,Set1,95.0,52,52,52,68,42,42,42,42,42,68,68,52
1,Set2,75.0,0,0,0,54,28,28,28,28,28,54,54,0
2,Set3,50.0,86,86,86,0,28,28,28,28,28,0,0,86
3,Set4,30.0,86,86,86,54,0,0,0,0,0,54,54,86
4,Set5,30.0,86,86,86,54,14,14,14,14,14,54,54,86
5,Set6,20.0,100,100,100,68,42,42,42,42,42,68,68,100


In [ ]:
#4.2 Визначити найкращу стратегію при поверненні протягом одного з 12-ти місяців за умови, що ймовірність повернення в кожен з місяців однакова.

# Функція для обчислення найкращої стратегії при рівноймовірному поверненні з максимізацією корисності
def calculate_best_strategy_maximization(sets, months_temperature, items):
    results = []

    # Ймовірність для кожного місяця
    probability = 1 / 12

    # Перебираємо кожен набір
    for set_name, set_data in sets.items():
        transportation_cost = set_data['weight'] * 10  # Вартість перевезення
        total_expected_utility = 0  # Очікувана корисність для цього набору

        # Перебір місяців
        for month, temp in enumerate(months_temperature):
            required_items, _, _ = get_best_set_for_month(temp, sets)  # Потрібні речі для цього місяця
            missing_cost, _ = calculate_missing_items_cost(set_data['items'], required_items, items)  # Вартість докупівлі
            month_cost = transportation_cost + missing_cost  # Загальні витрати для місяця
            total_expected_utility += -month_cost * probability

        # Зберігаємо результати для цього набору
        results.append({'Набір': set_name, 'Очікувана корисність': total_expected_utility})

    # Формуємо таблицю результатів
    return pd.DataFrame(results)

# Обчислюємо найкращу стратегію з максимізацією корисності
equal_probability_utility = calculate_best_strategy_maximization(sets, months_temperature, items)

# Знаходимо набір з максимальною очікуваною корисністю
best_strategy_maximization = equal_probability_utility.loc[equal_probability_utility['Очікувана корисність'].idxmax()]

# Виводимо результати
equal_probability_utility


,Набір,Очікувана корисність
0,Set1,-146.833333
1,Set2,-100.166667
2,Set3,-90.333333
3,Set4,-72.166667
4,Set5,-78.000000
5,Set6,-87.833333


In [ ]:
best_strategy_maximization

,3
Набір,Set4
Очікувана корисність,-72.166667


In [ ]:
#4.3 Визначити найкращу стратегію за при поверненні протягом одного сезону за наданих наборів ймовірностей повернення у кожен з місяців.


# Функція для обчислення найкращої стратегії по сезонах з максимізацією корисності
def calculate_best_strategy_by_season_maximization(sets, months_temperature, items, season_probabilities):
    results = []

    # Для кожного сезону
    for season, probabilities in season_probabilities.items():
        season_result = {'Сезон': season}

        # Для кожного набору
        for set_name, set_data in sets.items():
            transportation_cost = set_data['weight'] * 10  # Вартість перевезення
            total_expected_utility = 0  # Очікувана корисність для цього набору в цьому сезоні

            # Перебір місяців і ймовірностей для сезону
            for month, prob in enumerate(probabilities):
                if prob > 0:  # Якщо ймовірність повернення у місяць більше 0
                    temp = months_temperature[month]
                    required_items, _, _ = get_best_set_for_month(temp, sets)  # Потрібні речі для цього місяця
                    missing_cost, _ = calculate_missing_items_cost(set_data['items'], required_items, items)  # Вартість докупівлі
                    month_cost = transportation_cost + missing_cost  # Загальні витрати для місяця
                    total_expected_utility += -month_cost * prob

            # Зберігаємо результат для цього набору
            season_result[set_name] = total_expected_utility

        results.append(season_result)

    # Формуємо таблицю результатів
    return pd.DataFrame(results)

# Обчислюємо очікувану корисність для кожного сезону
season_probabilities = {
    'Зима': [1/3, 1/3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1/3],  # Грудень, Січень, Лютий
    'Весна': [0, 0, 1/3, 1/3, 1/3, 0, 0, 0, 0, 0, 0, 0],  # Березень, Квітень, Травень
    'Літо': [0, 0, 0, 0, 0, 1/3, 1/3, 1/3, 0, 0, 0, 0],  # Червень, Липень, Серпень
    'Осінь': [0, 0, 0, 0, 0, 0, 0, 0, 1/3, 1/3, 1/3, 0]   # Вересень, Жовтень, Листопад
}

# Результати для кожного сезону
season_utility_results = calculate_best_strategy_by_season_maximization(sets, months_temperature, items, season_probabilities)

# Знаходимо найкращу стратегію (максимальна корисність) для кожного сезону
best_strategy_by_season = season_utility_results.set_index('Сезон').idxmax(axis=1)

# Виводимо таблицю корисності та найкращі стратегії
season_utility_results


,Сезон,Set1,Set2,Set3,Set4,Set5,Set6
0,Зима,-147.000000,-75.000000,-136.000000,-116.000000,-116.000000,-120.000000
1,Весна,-149.000000,-102.333333,-88.000000,-76.666667,-81.333333,-90.000000
2,Літо,-137.000000,-103.000000,-78.000000,-30.000000,-44.000000,-62.000000
3,Осінь,-154.333333,-120.333333,-59.333333,-66.000000,-70.666667,-79.333333


In [ ]:
best_strategy_by_season


,0
Сезон,
Зима,Set2
Весна,Set4
Літо,Set4
Осінь,Set3


In [ ]:
#4.4 Визначити найкращу стратегію при поверненні протягом одного з 12-ти місяців за умови, що ймовірність повернення взимку втричі більша за інші місяці.

# Ймовірності повернення в кожен місяць
month_probabilities = [
    1/6,  # Грудень
    1/6,  # Січень
    1/6,  # Лютий
    1/18, # Березень
    1/18, # Квітень
    1/18, # Травень
    1/18, # Червень
    1/18, # Липень
    1/18, # Серпень
    1/18, # Вересень
    1/18, # Жовтень
    1/18  # Листопад
]
# Функція для розрахунку найкращої стратегії з врахуванням ймовірностей повернення
def calculate_best_strategy_custom_probabilities(sets, months_temperature, items, month_probabilities):
    results = []

    # Перебираємо кожен набір
    for set_name, set_data in sets.items():
        transportation_cost = set_data['weight'] * 10  # Вартість перевезення
        total_expected_utility = 0  # Очікувана корисність для цього набору

        # Перебір місяців
        for month, temp in enumerate(months_temperature):
            prob = month_probabilities[month]  # Ймовірність для цього місяця
            required_items, _, _ = get_best_set_for_month(temp, sets)  # Потрібні речі для цього місяця
            missing_cost, _ = calculate_missing_items_cost(set_data['items'], required_items, items)  # Вартість докупівлі
            month_cost = transportation_cost + missing_cost  # Загальні витрати для місяця
            total_expected_utility += -month_cost * prob  # Перетворюємо витрати на "негативну корисність"

        # Зберігаємо результати для цього набору
        results.append({'Набір': set_name, 'Очікувана корисність': total_expected_utility})

    # Формуємо таблицю результатів
    return pd.DataFrame(results)

# Обчислюємо результати
custom_probability_results = calculate_best_strategy_custom_probabilities(sets, months_temperature, items, month_probabilities)

# Знаходимо найкращий набір (максимальна корисність)
best_strategy_custom_probabilities = custom_probability_results.loc[custom_probability_results['Очікувана корисність'].idxmax()]

# Виводимо результати
custom_probability_results


,Набір,Очікувана корисність
0,Set1,-146.888889
1,Set2,-91.777778
2,Set3,-105.555556
3,Set4,-86.777778
4,Set5,-90.666667
5,Set6,-98.555556


In [ ]:
best_strategy_custom_probabilities

,3
Набір,Set4
Очікувана корисність,-86.777778


In [ ]:
#4.5 Визначити найкращу стратегію при поверненні протягом одного з 12-ти місяців за умови, що ймовірність повернення залежить від кількості днів у місяці (рік вважати не високосним).

# Кількість днів у кожному місяці
days_in_month = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

# Розрахунок ймовірностей для кожного місяця
month_probabilities = [days / 365 for days in days_in_month]

# Функція для розрахунку найкращої стратегії з урахуванням ймовірностей за кількістю днів у місяці
def calculate_best_strategy_days_in_month(sets, months_temperature, items, month_probabilities):
    results = []

    # Перебираємо кожен набір
    for set_name, set_data in sets.items():
        transportation_cost = set_data['weight'] * 10  # Вартість перевезення
        total_expected_utility = 0  # Очікувана корисність для цього набору

        # Перебір місяців
        for month, temp in enumerate(months_temperature):
            prob = month_probabilities[month]  # Ймовірність для цього місяця
            required_items, _, _ = get_best_set_for_month(temp, sets)  # Потрібні речі для цього місяця
            missing_cost, _ = calculate_missing_items_cost(set_data['items'], required_items, items)  # Вартість докупівлі
            month_cost = transportation_cost + missing_cost  # Загальні витрати для місяця
            total_expected_utility += -month_cost * prob  # Перетворюємо витрати на "негативну корисність"

        # Зберігаємо результати для цього набору
        results.append({'Набір': set_name, 'Очікувана корисність': total_expected_utility})

    # Формуємо таблицю результатів
    return pd.DataFrame(results)

# Розрахунок найкращої стратегії
best_strategy_days_in_month_results = calculate_best_strategy_days_in_month(sets, months_temperature, items, month_probabilities)

# Знаходимо найкращий набір (максимальна корисність)
best_strategy_days_in_month = best_strategy_days_in_month_results.loc[best_strategy_days_in_month_results['Очікувана корисність'].idxmax()]

# Виводимо результати
best_strategy_days_in_month_results


,Набір,Очікувана корисність
0,Set1,-146.797260
1,Set2,-100.200000
2,Set3,-90.246575
3,Set4,-71.972603
4,Set5,-77.841096
5,Set6,-87.709589


In [ ]:
best_strategy_days_in_month

,3
Набір,Set4
Очікувана корисність,-71.972603


In [ ]:
# 4.6 Визначити найкращу стратегію при поверненні протягом одного з 12-ти місяців за умови,
# що ймовірність повернення в кожен з місяців однакова, а початкова вартість речей з номерами № 2, 4, 9, 15, 16  зменшилася втричі.



# Оновлюємо вартість речей
items_to_update = ['Зимова куртка', 'В’єтнамки', 'Пальто', 'Чоботи', 'Шапка']

for item in items_to_update:
    items[item]['cost'] = items[item]['cost'] / 3

# Функція для розрахунку найкращої стратегії при рівноймовірному поверненні з оновленими цінами
def calculate_best_strategy_updated_costs(sets, months_temperature, items):
    results = []

    # Ймовірність для кожного місяця
    probability = 1 / 12

    # Перебираємо кожен набір
    for set_name, set_data in sets.items():
        transportation_cost = set_data['weight'] * 10  # Вартість перевезення
        total_expected_utility = 0  # Очікувана корисність для цього набору

        # Перебір місяців
        for month, temp in enumerate(months_temperature):
            required_items, _, _ = get_best_set_for_month(temp, sets)  # Потрібні речі для цього місяця
            missing_cost, _ = calculate_missing_items_cost(set_data['items'], required_items, items)  # Вартість докупівлі
            month_cost = transportation_cost + missing_cost  # Загальні витрати для місяця
            total_expected_utility += -month_cost * probability  # Перетворюємо витрати на "негативну корисність"

        # Зберігаємо результати для цього набору
        results.append({'Набір': set_name, 'Очікувана корисність': total_expected_utility})

    # Формуємо таблицю результатів
    return pd.DataFrame(results)

# Обчислюємо результати
updated_costs_results = calculate_best_strategy_updated_costs(sets, months_temperature, items)

# Знаходимо найкращий набір (максимальна корисність)
best_strategy_updated_costs = updated_costs_results.loc[updated_costs_results['Очікувана корисність'].idxmax()]

# Виводимо результати
updated_costs_results


,Набір,Очікувана корисність
0,Set1,-138.833333
1,Set2,-100.166667
2,Set3,-74.333333
3,Set4,-56.166667
4,Set5,-62.000000
5,Set6,-71.833333


In [ ]:
best_strategy_updated_costs

,3
Набір,Set4
Очікувана корисність,-56.166667
